<a href="https://colab.research.google.com/github/jogabenn/Actividades-MNA-Inteligencia-Artificial-y-Aprendizaje-Automatico/blob/main/Actividades%20Semana%2010/SVD_Equipo50.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Maestría en Inteligencia Artificial Aplicada**
## **Curso: Inteligencia Artificial y Aprendizaje Automático**
### Tecnológico de Monterrey
### Prof. Luis Eduardo Falcón Morales

## **Actividad de la Semana 10**
### **Reducción de Dimensionalidad - Descomposición en Valores Singulares (SVD) y Sistemas de Recomendación**


**Nombres y matrículas de los integrantes del equipo:**

* Jonathan Garza Bennet | A01793038
* Diego Fernando Guerra Burgos | A01793499
* Fernando Maytorena Espinosa de los Monteros | A01110946  


---
Esta Tarea se deberá resolver en equipos, de acuerdo a como fueron integrados al inicio del curso. Se trabajará con el conjunto de datos llamado "Restaurant & consumer", para la generación de un sistema de recomendación. Estos sistemas están dentro de los llamados métodos no supervisados y que se basan en técnicas de reducción de dimensionalidad. El archivo a descargar se llama RCdata.zip y se encuentra en la siguiente liga del repositorio de la UCI:
https://archive.ics.uci.edu/ml/datasets/Restaurant+%26+consumer+data<br>  
En particular se trabajará con los siguientes archivos, los cuales contienen la información mínima que requerimos para esta actividad: *rating_final.csv* y *geoplaces2.csv*.<br><br>
El objetivo es diseñar tres sistemas de recomendación basados en la evaluación de los usarios a los restaurantes. Dicha evaluación será la general, la referente al servicio y la referente a la comida. 

In [233]:
%%capture
!pip install fsspec

In [234]:
import pandas as pd
import numpy as np
from sklearn.decomposition import TruncatedSVD

### **Ejercicio 1**
Descarga los archivos y genera la matriz de utilidad en la cual los renglones son los usuarios (user_ID), las columnas los nombres de los restaurantes (name) y las entradas la evaluaciones generales dadas por los usuarios (rating). En particular indica el tamaño de dicha matriz y despliega sus primeros 10 renglones.

In [235]:
#URL conjunto de datos de evaluaciones
url= 'https://raw.githubusercontent.com/jogabenn/Actividades-MNA-Inteligencia-Artificial-y-Aprendizaje-Automatico/main/Actividades%20Semana%2010/rating_final.txt'

#Asignar a un dataframe pandas
df_rf = pd.read_csv(url, sep=',', header=0)


In [236]:
#Se imprime forma del conjunto de datos
print(df_rf.shape)

#Se imprime encabezado
df_rf.head()

(1161, 5)


,userID,placeID,rating,food_rating,service_rating
0,U1077,135085,2,2,2
1,U1077,135038,2,2,1
2,U1077,132825,2,2,2
3,U1077,135060,1,2,2
4,U1068,135104,1,1,2


In [237]:
#URL conjunto de datos de ubicaciones
url= 'https://raw.githubusercontent.com/jogabenn/Actividades-MNA-Inteligencia-Artificial-y-Aprendizaje-Automatico/main/Actividades%20Semana%2010/geoplaces2.txt'

#Asignar a un dataframe pandas
df_gp = pd.read_csv(url, delimiter=',', header=0, encoding='latin-1')

In [238]:
#Se imprime forma del conjunto de datos
print(df_gp.shape)

#Se imprime encabezado
df_gp.head()

(130, 21)


,placeID,latitude,longitude,the_geom_meter,name,address,city,state,country,fax,...,alcohol,smoking_area,dress_code,accessibility,price,url,Rambience,franchise,area,other_services
0,134999,18.915421,-99.184871,0101000020957F000088568DE356715AC138C0A525FC46...,Kiku Cuernavaca,Revolucion,Cuernavaca,Morelos,Mexico,?,...,No_Alcohol_Served,none,informal,no_accessibility,medium,kikucuernavaca.com.mx,familiar,f,closed,none
1,132825,22.147392,-100.983092,0101000020957F00001AD016568C4858C1243261274BA5...,puesto de tacos,esquina santos degollado y leon guzman,s.l.p.,s.l.p.,mexico,?,...,No_Alcohol_Served,none,informal,completely,low,?,familiar,f,open,none
2,135106,22.149709,-100.976093,0101000020957F0000649D6F21634858C119AE9BF528A3...,El Rincón de San Francisco,Universidad 169,San Luis Potosi,San Luis Potosi,Mexico,?,...,Wine-Beer,only at bar,informal,partially,medium,?,familiar,f,open,none
3,132667,23.752697,-99.163359,0101000020957F00005D67BCDDED8157C1222A2DC8D84D...,little pizza Emilio Portes Gil,calle emilio portes gil,victoria,tamaulipas,?,?,...,No_Alcohol_Served,none,informal,completely,low,?,familiar,t,closed,none
4,132613,23.752903,-99.165076,0101000020957F00008EBA2D06DC8157C194E03B7B504E...,carnitas_mata,lic. Emilio portes gil,victoria,Tamaulipas,Mexico,?,...,No_Alcohol_Served,permitted,informal,completely,medium,?,familiar,t,closed,none


In [239]:
#Se combinan los conjuntos de datos y se asignan a un nuevo dataframe
df1 = pd.merge(df_rf, df_gp, on='placeID') 

#Se utilizan sólo las columnas 'userID', 'name' y 'rating'
df1 = df1[['userID', 'name', 'rating']]

#Se imprime encabezado
df1.head()

,userID,name,rating
0,U1077,Tortas Locas Hipocampo,2
1,U1108,Tortas Locas Hipocampo,1
2,U1081,Tortas Locas Hipocampo,1
3,U1056,Tortas Locas Hipocampo,2
4,U1134,Tortas Locas Hipocampo,2


In [240]:
#Se obtiene la matriz de utilidad con 'userID' como renglones y los nombres de los restaurantes como columnas
UtMx = df1.pivot_table(values='rating', index='userID', columns='name', fill_value=0)

#Se imprime la forma de la matriz y los primeros 10 renglones
print('La matriz de utilidad consta de {0} filas y {1} columnas: {2}'.format(UtMx.shape[0], UtMx.shape[1], (UtMx.columns[0], UtMx.columns[1], UtMx.columns[2])))
print("A continuación se muestran los primeros 10 renglones de la matriz de utilidad:")
UtMx.head(10)

La matriz de utilidad consta de 138 filas y 129 columnas: ('Abondance Restaurante Bar', 'Arrachela Grill', 'Cabana Huasteca')
A continuación se muestran los primeros 10 renglones de la matriz de utilidad:


name,Abondance Restaurante Bar,Arrachela Grill,Cabana Huasteca,Cafe Chaires,Cafeteria cenidet,Cafeteria y Restaurant El Pacifico,Carls Jr,Carnitas Mata Calle 16 de Septiembre,Carreton de Flautas y Migadas,Cenaduria El RincÃ³n de Tlaquepaque,...,puesto de tacos,rockabilly,shi ro ie,sirloin stockade,tacos abi,tacos de barbacoa enfrente del Tec,tacos de la estacion,tacos los volcanes,tortas hawai,vips
userID,,,,,,,,,,,,,,,,,,,,,
U1001,0,0,0,0,0,0,0,0,0,0,...,2,0,0,0,0,0,0,0,0,0
U1002,0,0,0,0,0,0,0,0,0,0,...,2,0,0,0,0,0,0,0,0,0
U1003,0,0,2,0,0,0,0,0,0,0,...,2,2,0,0,0,0,0,0,0,0
U1004,0,0,0,0,0,2,0,0,0,0,...,0,0,0,0,0,0,0,2,0,0
U1005,0,0,0,0,0,2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
U1006,0,0,0,1,0,0,0,0,0,0,...,1,0,2,0,0,0,0,0,0,0
U1007,0,0,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
U1008,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
U1009,0,0,1,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0


### **Ejercicio 2**
Con el objetivo de obtener la información con respecto a los vectores latentes de los restaurantes, obtener dicha factorización SVD utilizando todas las componentes posibles, es decir, usando todos sus valores singulares. En particular indica la cantidad de valores singulares obtenidos.

In [241]:
#Se obtiene la transpuesta de la matriz de utilidad para que las filas sean los restaurantes
#ya que la información de importancia que deseamos obtener es sobre la relación entre los restaurantes y usuarios
X = UtMx.T

#Se imprime la forma de la matriz
X.shape

(129, 138)

In [242]:
#El número de filas -1 para realizar la factorización SVD
X.shape[1]-1

137

In [243]:
# Obtengamos la cantidad de información contenida en la factorización SVD, considerando 
# los valores singulares más representativos de dicha factorización.
# Para este caso inicial usaremos todos los vectores singulares, para luego 
# compararlo con el caso al reducir su dimensión.

num_sv = X.shape[1]-1

# El número de componentes debe ser menor que el número de características:
SVD = TruncatedSVD(n_components=(X.shape[1]-1), random_state=1)  

#Se realiza la factorización SVD
SVD.fit(X)

#Se imprimen los resultados
print('Cantidad de información simplificada con los primeros %d vectores singulares:' % num_sv)
print('%.1f%%' %  (100 * (1- (SVD.singular_values_[0:num_sv].sum() / SVD.singular_values_.sum()))))
print('Después de aplicar la factorización SVD, obtenemos {0} valores singulares'.format(len(SVD.singular_values_)))

Cantidad de información simplificada con los primeros 137 vectores singulares:
0.0%
Después de aplicar la factorización SVD, obtenemos 129 valores singulares


### **Ejercicio 3**
Usando la métrica de correlación de Pearson, obtener las 10 mejores recomendaciones de este modelo no supervisado con base a la información de alguien que evaluó bien de manera general a “Subway”. Despliega también el valor numérico de la correlación de Pearson de cada una de estas 10 recomendaciones. NOTA: No debe estar incluido “Subway” dentro de las recomendaciones.

In [244]:
# El número de componentes debe ser menor que el número de características:
SVD = TruncatedSVD(num_sv, random_state=1) 

#Con la factorización realizada anteriormente, se transforma la matriz X
resultant_matrix = SVD.fit_transform(X)

#Se imprime la forma
print(resultant_matrix.shape)

#Se crea un dataframe para desplegar el encabezado
rm = pd.DataFrame(resultant_matrix)
rm.head().round(2)

(129, 129)


,0,1,2,3,4,5,6,7,8,9,...,119,120,121,122,123,124,125,126,127,128
0,0.22,-0.34,0.93,-0.00,0.08,0.77,0.43,-0.43,0.13,0.09,...,0.0,-0.01,0.01,-0.03,-0.00,-0.01,0.0,0.01,0.0,-0.0
1,-0.00,0.00,0.00,0.12,0.00,0.00,-0.00,0.00,-0.00,0.00,...,0.0,-0.00,0.00,-0.00,-0.02,-0.00,0.0,-0.00,0.0,-0.0
2,3.05,2.56,-0.48,-0.00,1.03,1.39,0.47,0.87,0.57,-1.52,...,-0.0,0.05,0.01,-0.02,0.00,-0.01,0.0,-0.00,-0.0,0.0
3,1.23,1.65,0.67,0.00,-1.58,0.30,-0.27,0.57,0.25,1.21,...,-0.0,0.02,-0.04,0.03,-0.00,-0.01,0.0,0.01,0.0,-0.0
4,-0.00,0.00,0.00,0.90,0.00,0.00,-0.00,0.00,-0.00,0.00,...,-0.0,-0.00,0.00,0.00,0.01,-0.00,0.0,0.00,0.0,-0.0


In [245]:
#Se calcula el coeficiente de correlación de pearson entre todos los vectores
corrMx = np.corrcoef(resultant_matrix, )

#Se imprime la forma de la matriz de correlación
print(corrMx.shape)

#Se crea un dataframe para desplegar el encabezado
cm = pd.DataFrame(corrMx)
cm.head().round(3)

(129, 129)


,0,1,2,3,4,5,6,7,8,9,...,119,120,121,122,123,124,125,126,127,128
0,1.000,-0.000,-0.004,0.001,-0.002,0.162,-0.001,-0.004,0.001,-0.001,...,0.046,-0.002,-0.000,-0.002,0.001,0.002,-0.002,-0.002,-0.001,0.001
1,-0.000,1.000,-0.000,0.000,0.317,0.000,-0.000,-0.000,0.000,-0.000,...,-0.000,-0.000,-0.000,-0.000,0.000,0.000,-0.000,-0.000,-0.000,0.000
2,-0.004,-0.000,1.000,0.229,-0.016,0.046,-0.008,-0.028,0.004,-0.008,...,0.431,0.205,0.044,-0.017,0.010,0.013,-0.014,0.147,-0.010,0.004
3,0.001,0.000,0.229,1.000,0.003,0.087,0.165,0.005,-0.001,0.001,...,0.160,0.002,0.338,0.003,-0.002,-0.002,0.002,0.106,0.002,-0.001
4,-0.002,0.317,-0.016,0.003,1.000,0.000,-0.005,-0.016,0.002,-0.005,...,-0.011,-0.007,-0.000,-0.010,0.006,0.008,-0.008,-0.009,-0.006,0.003


Se buscan las recomenaciones para el restaurante "Subway"

In [246]:
#Nombre del restaurante para buscar recomendaciones
me_gusta = "Subway"

#Se extraen los nombres de las columnas de la matriz de utilidad y se asignan a una lista
names = UtMx.columns
names_list = list(names)

#Se extraen los índices de la matriz de utilidad y se asignan a una lista
index = np.linspace(0, len(names_list)-1, len(names_list), dtype=int)
index_list = list(index)

#Se extrae el índice del restaurante
id_megusta = names_list.index(me_gusta)

#Se imprime el índice del restaurante
print('El índice de '+me_gusta+' es: {}'.format(id_megusta))

El índice de Subway es: 88


In [247]:
#Se obtiene la correlación del restaurante con todos los del catálogo
corr_recomienda = corrMx[id_megusta]

#Se imprime la forma y los resultados
print(corr_recomienda.shape)
print(corr_recomienda)

(129,)
[-7.69028369e-04 -7.87822168e-05 -5.27330975e-03  8.58511349e-04
 -3.03891148e-03  3.97495832e-05 -1.54820208e-03 -5.29347145e-03
  7.27520383e-04 -1.55098452e-03 -1.84483094e-03  3.01090491e-01
 -3.06938616e-05 -3.66639157e-03 -2.28024815e-03 -1.70971180e-03
  2.66961340e-03  4.58467365e-04 -2.11449533e-03 -6.20445134e-03
  3.34268563e-03  1.16598607e-01  1.87080102e-03  2.53682556e-03
 -5.48801141e-03 -2.91288266e-03  2.02825342e-03 -1.36607265e-03
 -2.01695899e-03  1.12058244e-01 -2.36235727e-03  6.52159239e-04
  3.53495770e-03 -3.62793330e-03  1.69065407e-03 -4.51837401e-03
 -2.09301342e-03  2.34634298e-01 -3.80413494e-03 -5.11845710e-04
 -2.27148704e-03  3.88178002e-01 -2.07427108e-03 -2.29428835e-03
  1.32565455e-03  3.01002479e-01  1.71433518e-01 -3.29715354e-03
 -4.74480596e-03 -8.02527884e-04 -2.65071452e-03  1.12018889e-01
 -3.91080724e-03 -2.06369812e-03  1.16008629e-01  2.27855804e-01
  4.28342495e-04 -8.69739761e-04  6.09520875e-01 -6.43364064e-03
  4.25467169e-01 -

In [248]:
#Se obtiene la correlación consigo mismo para verificar los resultados
corr_recomienda[id_megusta]

1.0

In [249]:
# Primero, encontramos los restaurantes (por nombre) que tienen la mayor correlación con el restaurante Subway,
# para ello, filtramos a los restaurantes con correlación mayor a 0.20, y menor a 1 (para eliminar al mismo Subway de la lista):
subway_similar_names = list(names[(corr_recomienda > .20) & (corr_recomienda < 1)])
#print(subway_similar_names)

# Después, obtenemos los índices de los restaurantes de la lista que acabamos de obtener:
subway_similar_index = list()
for i in range(len(subway_similar_names)):
    subway_similar_index.append(names_list.index(subway_similar_names[i]))
#print(subway_similar_index)


# Luego, obtenemos los valores de la correlación de la misma lista que acabamos de obtener:
subway_similar_corr = list()
for i in range(len(subway_similar_index)):
    subway_similar_corr.append(corr_recomienda[subway_similar_index[i]])
#print(subway_similar_corr)

# Finalmente, creamos un DataFrame con los nombres de los restaurantes, sus índices y sus valores de correlación,
# los ordenamos de mayor a menor de acuerdo con su correlación, y obtenemos el top 10 de restaurantes recomendados para usuarios que les gusta el restaurante Subway:
subway_similar = pd.DataFrame({'name': subway_similar_names, 'index': subway_similar_index, 'corr': subway_similar_corr})
subway_similar = subway_similar.sort_values(by=['corr'], ascending=False)
subway_similar.head(10)

,name,index,corr
5,Restaurant Teely,58,0.609521
10,Vips,97,0.506278
8,Rincon del Bife,85,0.429824
6,Restaurant and Bar and Clothesline Carlos N Ch...,60,0.425467
9,Sanborns Casa Piedra,86,0.417219
2,Mariscos Tia Licha,41,0.388178
0,Chilis Cuernavaca,11,0.301090
3,Mikasa,45,0.301002
7,Restaurant los Pinos,64,0.285149
1,Log Yin,37,0.234634


### **Ejercicio 4**
Disminuye ahora la dimensión de la factorización con un número de componentes tal, que las recomendaciones con esta menor cantidad de información coincidan en al menos 9 de las 10 recomendaciones dadas con todos los valores singulares. Indica la cantidad de valores singulares encontrado y la cantidad de información que contienen.

In [292]:
# Obtengamos la cantidad de información contenida en la factorización SVD, considerando 
# los valores singulares más representativos de dicha factorización.
# Para este caso inicial usaremos todos los vectores singulares, para luego 
# compararlo con el caso al reducir su dimensión.

factor_de_reduccion = 0.21

num_sv = round((X.shape[1]-1)*factor_de_reduccion)

#Se imprimen los resultados
print('Cantidad de información que contienen los primeros {} '.format(num_sv) + 
      'vectores singulares es: {}'.format(round(100 * ((SVD.singular_values_[0:num_sv].sum() / SVD.singular_values_.sum())),2))+'%\n')
#print('%.1f%%\n' %  (100 * ((SVD.singular_values_[0:num_sv].sum() / SVD.singular_values_.sum()))))

# El número de componentes debe ser menor que el número de características:
SVD_red = TruncatedSVD(num_sv, random_state=1) 

#Con la factorización realizada anteriormente, se transforma la matriz X
resultant_matrix_red = SVD_red.fit_transform(X)

print('Después de aplicar la factorización SVD Reducida, obtenemos {0} valores singulares'.format(len(SVD_red.singular_values_))+'\n')

#Se imprime la forma
#print(resultant_matrix_red.shape)

#Se crea un dataframe para desplegar el encabezado
rm_red = pd.DataFrame(resultant_matrix_red)
rm_red.head().round(2)

#Se calcula el coeficiente de correlación de pearson entre todos los vectores
corrMx_red = np.corrcoef(resultant_matrix_red, )

#Se imprime la forma de la matriz de correlación
#print(corrMx_red.shape)

#Se crea un dataframe para desplegar el encabezado
cm_red = pd.DataFrame(corrMx_red)
cm_red.head().round(3)

#Nombre del restaurante para buscar recomendaciones
me_gusta_red = "Subway"

#Se extraen los nombres de las columnas de la matriz de utilidad y se asignan a una lista
names = UtMx.columns
names_list = list(names)

#Se extraen los índices de la matriz de utilidad y se asignan a una lista
index = np.linspace(0, len(names_list)-1, len(names_list), dtype=int)
index_list = list(index)

#Se extrae el índice del restaurante
id_megusta_red = names_list.index(me_gusta_red)

#Se imprime el índice del restaurante
print('El índice de '+me_gusta_red+' es: {}'.format(id_megusta_red)+'\n')

#Se obtiene la correlación del restaurante con todos los del catálogo
corr_recomienda_red = corrMx_red[id_megusta_red]

#Se imprime la forma y los resultados
#print(corr_recomienda_red.shape)
#print(corr_recomienda_red)

# Primero, encontramos los restaurantes (por nombre) que tienen la mayor correlación con el restaurante Subway,
# para ello, filtramos a los restaurantes con correlación mayor a 0.20, y menor a 1 (para eliminar al mismo Subway de la lista):
subway_similar_names_red = list(names[(corr_recomienda_red > .20) & (corr_recomienda_red < 1)])

# Después, obtenemos los índices de los restaurantes de la lista que acabamos de obtener:
subway_similar_index_red = list()
for i in range(len(subway_similar_names_red)):
    subway_similar_index_red.append(names_list.index(subway_similar_names_red[i]))

# Luego, obtenemos los valores de la correlación de la misma lista que acabamos de obtener:
subway_similar_corr_red = list()
for i in range(len(subway_similar_index_red)):
    subway_similar_corr_red.append(corr_recomienda_red[subway_similar_index_red[i]])
#print(subway_similar_corr)

# Finalmente, creamos un DataFrame con los nombres de los restaurantes, sus índices y sus valores de correlación,
# los ordenamos de mayor a menor de acuerdo con su correlación, y obtenemos el top 10 de restaurantes recomendados para usuarios que les gusta el restaurante Subway:
subway_similar_red = pd.DataFrame({'name': subway_similar_names_red, 'index': subway_similar_index_red, 'corr': subway_similar_corr_red})
subway_similar_red = subway_similar_red.sort_values(by=['corr'], ascending=False)

elementos_iguales = 0

for nombre in subway_similar_red['name'].head(10):

  encontrar_nombre = subway_similar['name'].head(10) == nombre

  elementos_iguales = elementos_iguales + encontrar_nombre.sum()

print('La lista de recomendaciones original es:\n\n{}'.format(subway_similar.head(10))+'\n')

print('La lista de recomendaciones después de la reducción de dimensiones es:\n\n{}'.format(subway_similar_red.head(10))+'\n')

print('La cantidad de elementos iguales es: {}'.format(elementos_iguales))

Cantidad de información que contienen los primeros 29 vectores singulares es: 51.86%

Después de aplicar la factorización SVD Reducida, obtenemos 29 valores singulares

El índice de Subway es: 88

La lista de recomendaciones original es:

                                                 name  index      corr
5                                    Restaurant Teely     58  0.609521
10                                               Vips     97  0.506278
8                                     Rincon del Bife     85  0.429824
6   Restaurant and Bar and Clothesline Carlos N Ch...     60  0.425467
9                                Sanborns Casa Piedra     86  0.417219
2                                  Mariscos Tia Licha     41  0.388178
0                                   Chilis Cuernavaca     11  0.301090
3                                              Mikasa     45  0.301002
7                                Restaurant los Pinos     64  0.285149
1                                             Log Y

### **Ejercicio 5**
Repite el ejercicio con el restaurante “"tacos de barbacoa enfrente del Tec". Incluye tus conclusiones. 

### **Ejercicio 6**
Repite los ejercicios 1 a 4 generando ahora la matriz de utilidad en la cual los renglones son los usuarios ($user\_ID$), las columnas los nombres de los restaurantes ($name$) y las entradas la evaluación dada a la comida ($food\_rating$). Incluye tus conclusiones.

### **Ejercicio 7**
Repite los ejercicios 1 a 4 generando ahora la matriz de utilidad en la cual los renglones son los usuarios ($user\_ID$), las columnas los nombres de los restaurantes ($name$) y las entradas la evaluación dada al servicio ($service\_rating$). Incluye tus conclusiones. 

### **Ejercicio 8**
Incluye tus conclusiones finales. En particular indica la coincidencia o no de estos tres modelos. 

---